In [102]:
import sys
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, make_scorer
from imblearn.over_sampling import RandomOverSampler
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.cluster import SpectralClustering
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression


#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

sys.path.insert(0, '/Users/gracewang/Documents/GitHub/elecfinal')
sys.path.insert(0, 'D:\Fall23 Coursework\ELEC478\Competition\elecfinal')
from ml_pipeline import train_n_predict, validation, clean_split
from Data.data_cleaner import cleaner

# Clean Data

In [92]:
## Clean data
train_path = "../Data/train_data.csv"
feature_path = "../Data/feature_weights.csv"
morph_path = "../Data/imputed_morph_embed.csv"
X_train, X_val, X_query, y_train, y_val, y_query = clean_split(train_path, feature_path, morph_path)

In [93]:
X_train_feat = X_train.copy()
X_val_feat = X_val.copy()
X_query_feat = X_query.copy()

# Identify Most Important FW

In [80]:
X_train_fw = X_train.copy() # make a copy of the original X_train

In [81]:
# get a dataframe of feature weights for each obs. in X
fw = pd.read_csv(feature_path)
X_train_fw = fw.merge(X_train_fw['pre_nucleus_id'], right_on ='pre_nucleus_id', left_on='nucleus_id')
fw

,nucleus_id,feature_weight_0,feature_weight_1,feature_weight_2,feature_weight_3,feature_weight_4,feature_weight_5,feature_weight_6,feature_weight_7,feature_weight_8,...,feature_weight_502,feature_weight_503,feature_weight_504,feature_weight_505,feature_weight_506,feature_weight_507,feature_weight_508,feature_weight_509,feature_weight_510,feature_weight_511
0,557058,0.205733,0.317099,-0.241713,0.014264,-0.314002,-0.432338,0.152562,0.186511,0.140944,...,-0.034500,0.363631,0.183755,0.231815,0.042877,0.402666,0.016072,0.072968,-0.452475,-0.175632
1,155650,0.166665,-0.293123,-0.017896,-0.159223,0.246045,-0.138767,0.217365,-0.174787,-0.050760,...,0.276523,0.333738,0.277954,-0.124248,0.228173,0.029465,0.198626,-0.044919,0.661572,0.110016
2,425987,0.453440,0.174334,0.279131,0.144052,0.218536,0.110859,-0.090576,0.125406,0.110690,...,0.300525,-0.160008,0.052873,0.112233,-0.091779,-0.046790,0.280856,0.399340,0.305763,0.218547
3,262149,0.537410,0.165957,0.543808,-0.386137,-0.113595,0.046351,0.054061,0.247789,-0.502975,...,-0.113184,0.117278,0.011819,0.491052,-0.161192,0.868506,0.215711,0.108492,-0.607824,0.296478
4,557067,-0.193376,-0.058421,-0.074607,0.009510,-0.038007,0.011739,-0.141879,0.308456,0.392383,...,0.327086,0.413605,-0.092548,-0.091461,-0.025144,-0.202813,0.241059,0.014646,-0.073372,-0.242132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,368622,-0.330541,0.475290,0.010314,0.532330,-0.475708,0.443309,0.129722,-0.246515,0.317328,...,-0.000365,-0.001144,0.316298,0.075985,0.049807,-0.024745,0.322744,-0.502619,0.218220,-0.048334
2688,614382,0.275322,0.435353,0.091335,-0.033993,-0.504363,-0.133884,-0.367399,0.478917,0.033984,...,0.121871,-0.017587,0.141196,0.140291,0.354196,-0.044375,0.495297,0.369650,0.114305,0.281917
2689,557041,-0.598981,0.061932,-0.074026,0.171314,-0.250348,0.438974,-0.235911,0.036702,0.365656,...,0.039513,-0.141897,-0.232804,0.020774,0.542075,0.380731,0.189838,0.150242,-0.222899,-0.032958
2690,196596,0.024009,0.075810,0.314906,-0.302266,0.007095,0.024825,0.432788,-0.238054,0.210615,...,0.218091,0.377625,0.157004,0.265006,0.412147,0.088211,0.780171,0.079002,-0.009118,0.244956


In [82]:
ros = SMOTE(random_state=0, sampling_strategy = 'minority')
X_train_fw, y_train_fw = ros.fit_resample(
        X_train_fw, y_train)

In [90]:
X_val_fw = X_val.copy()
X_val_fw = fw.merge(X_val_fw['pre_nucleus_id'], right_on ='pre_nucleus_id', left_on='nucleus_id')

rf = BalancedRandomForestClassifier(random_state=1)
rf.fit(X_train_fw.drop(columns=['pre_nucleus_id', 'nucleus_id']), y_train_fw)
preds = rf.predict(X_val_fw.drop(columns=['pre_nucleus_id', 'nucleus_id']))

print("RFC Validation Accuracy:", str(balanced_accuracy_score(y_val, preds)))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imblearn/ensemble/_forest.py:546: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imblearn/ensemble/_forest.py:558: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


RFC Validation Accuracy: 0.5


In [88]:
np.unique(y_train_fw)

array([False,  True])

In [33]:
imp_df = pd.DataFrame({"Features":X_val_fw.drop(columns=['pre_nucleus_id', 'nucleus_id']).columns, "Importance":rf.feature_importances_})
imp_df.sort_values('Importance', ascending=False)

,Features,Importance
101,feature_weight_101,0.064192
478,feature_weight_478,0.054274
194,feature_weight_194,0.046692
138,feature_weight_138,0.037233
84,feature_weight_84,0.035156
...,...,...
459,feature_weight_459,0.000025
10,feature_weight_10,0.000018
257,feature_weight_257,0.000014
363,feature_weight_363,0.000013


In [41]:
# Include FW 101 in the training data
def add_feature(df, fw, feature_name):
    """"
    Adds a feature weight to the dataframe
    """
    df = df.merge(fw[[feature_name, 'nucleus_id']], left_on='pre_nucleus_id', right_on='nucleus_id')
    return df

In [44]:
X_train_feat = add_feature(X_train_feat, fw, 'feature_weight_101')
X_val_feat = add_feature(X_val_feat, fw, 'feature_weight_101')
X_query_feat = add_feature(X_query_feat, fw, 'feature_weight_101')

# One-Hot Encode All Categorical Variables

In [94]:
def one_hot(column, df, suffix=''):
    """
    one-hot encodes this shit
    """
    cats = pd.unique(df[column])

    for cat in cats:
        new_col = cat+suffix
        df[new_col] = df[column]==cat
        df[new_col] = df[new_col].astype('int')
    
    df = df.drop(columns=column)
    return df

In [95]:
# one-hot encode brain areas for all
X_train_feat = one_hot('pre_brain_area', X_train, '_pre')
X_train_feat = one_hot('post_brain_area', X_train_feat, '_post')
X_train_feat = one_hot('projection_group', X_train_feat)

X_val_feat = one_hot('pre_brain_area', X_val, '_pre')
X_val_feat = one_hot('post_brain_area', X_val_feat, '_post')
X_val_feat = one_hot('projection_group', X_val_feat)

X_query_feat = one_hot('pre_brain_area', X_query, '_pre')
X_query_feat = one_hot('post_brain_area', X_query_feat, '_post')
X_query_feat = one_hot('projection_group', X_query_feat)

In [96]:
# encode brain areas
area1 = ["basal", "soma"]
area2 = ["axon", "apical", "oblique", "apical_shaft"]
area3 = ["apical_tuft"]

def area_cols(df):
    df["area1"] = df["compartment"].isin(area1).astype('int')
    df["area2"] = df["compartment"].isin(area2).astype('int')
    df["area3"] = df["compartment"].isin(area3).astype('int')
    df.drop(columns='compartment')
    return df

In [97]:
X_train_feat = area_cols(X_train_feat)
X_val_feat = area_cols(X_val_feat)
X_query_feat = area_cols(X_query_feat)

In [100]:
X_train_feat = X_train_feat.select_dtypes('number')
X_val_feat = X_val_feat.select_dtypes('number')
X_query_feat = X_query_feat.select_dtypes('number')

# Run Logistic Regression with Lasso For Feature Selection

In [101]:
# Oversample X_train_feat
ros = SMOTE(random_state=0, sampling_strategy = 'minority')
X_train_over, y_train_over = ros.fit_resample(
        X_train_feat, y_train)

In [108]:
X_train_over.columns

Index(['ID', 'adp_dist', 'post_skeletal_distance_to_soma',
       'pre_skeletal_distance_to_soma', 'pre_oracle', 'pre_test_score',
       'post_oracle', 'post_test_score', 'post_nucleus_x', 'post_nucleus_y',
       'post_nucleus_z', 'pre_nucleus_id', 'post_nucleus_id', 'me_similarity',
       'fw_similarity', 'nuclei_adp_dist', 'RL_pre', 'AL_pre', 'V1_pre',
       'RL_post', 'V1_post', 'AL_post', 'RL->RL', 'RL->V1', 'RL->AL', 'AL->V1',
       'AL->AL', 'AL->RL', 'V1->V1', 'V1->RL', 'V1->AL', 'area1', 'area2',
       'area3'],
      dtype='object')

In [109]:
X_val_feat.sort_index(axis=1)

,AL->AL,AL->RL,AL->V1,AL_post,AL_pre,ID,RL->AL,RL->RL,RL->V1,RL_post,...,post_nucleus_x,post_nucleus_y,post_nucleus_z,post_oracle,post_skeletal_distance_to_soma,post_test_score,pre_nucleus_id,pre_oracle,pre_skeletal_distance_to_soma,pre_test_score
9609,0,1,0,0,1,52202,0,0,0,1,...,0.472640,-1.113624,0.557703,1.191761,-0.251747,-0.205420,582210,-0.237200,1.326529,-0.813618
9610,0,1,0,0,1,52203,0,0,0,1,...,0.472640,-1.113624,0.557703,1.191761,5.002904,-0.205420,582210,-0.237200,1.124102,-0.813618
9611,0,0,1,0,1,52204,0,0,0,0,...,-1.345300,0.534638,-2.241494,-0.622637,-0.078685,-1.219202,582210,-0.237200,-1.714112,-0.813618
9612,0,0,1,0,1,52205,0,0,0,0,...,-1.345300,0.534638,-2.241494,-0.622637,-0.781174,-1.219202,582210,-0.237200,-1.813073,-0.813618
9613,0,0,1,0,1,52206,0,0,0,0,...,-1.345300,0.534638,-2.241494,-0.622637,0.096931,-1.219202,582210,-0.237200,0.385808,-0.813618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179187,0,0,0,0,0,221780,0,0,0,0,...,-0.923532,1.271107,0.041017,-0.024162,0.219202,-0.228849,294776,-0.298274,-1.030565,-1.628008
179188,0,0,0,0,0,221781,0,0,0,0,...,-0.923532,1.271107,0.041017,-0.024162,-0.855499,-0.228849,294776,-0.298274,-0.823973,-1.628008
179189,0,0,0,0,0,221782,0,0,0,0,...,-0.923532,1.271107,0.041017,-0.024162,1.371097,-0.228849,294776,-0.298274,-0.208724,-1.628008
179190,0,0,0,0,0,221783,0,0,0,0,...,-0.923532,1.271107,0.041017,-0.024162,-0.099739,-0.228849,294776,-0.298274,-0.818962,-1.628008


In [141]:
fw_model = LogisticRegression(max_iter=500, penalty='elasticnet', solver='saga', l1_ratio=0.2).fit(X_train_over.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1), y_train_over)
preds = fw_model.predict(X_val_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1))
balanced_accuracy_score(y_val,preds)

0.6733847791034687

In [143]:
feat_select = pd.DataFrame({"features":X_val_feat.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).columns})
feat_select['coefs']=abs(fw_model.coef_[0])
feat_select.sort_values(by='coefs', ascending=False)

,features,coefs
18,AL_post,9.539396
17,V1_post,9.142346
16,RL_post,7.843112
3,pre_oracle,4.742626
13,AL_pre,4.478484
14,RL_pre,3.744543
8,post_nucleus_y,3.725408
1,post_skeletal_distance_to_soma,3.459649
0,adp_dist,3.441890
6,post_test_score,3.414404


In [127]:
fw_model.coef_[0]

array([-4.14173929, -4.83687022, -2.67690186, -4.31208773, -2.3079184 ,
       -3.45444381, -4.69567364, -2.34620107, -2.61835907, -2.1470694 ,
       -2.87271978, -3.59890564, -1.48522776, -4.31131917, -3.19205854,
       -1.45149221, -7.77492608, -9.07915351, -9.50933349,  0.30014125,
        0.10871177,  0.23882281, -0.09650387, -0.01518531, -0.15637745,
       -0.07192351, -0.29332863, -0.01366229,  0.08671553,  0.22521566,
        0.18615776])

# Select Top 5 Fucking Features

In [146]:
top_feature = np.array(feat_select["features"][0:5])

In [147]:
top_feature

array(['adp_dist', 'post_skeletal_distance_to_soma',
       'pre_skeletal_distance_to_soma', 'pre_oracle', 'pre_test_score'],
      dtype=object)

In [149]:
X_train_over = X_train_over[top_feature]
X_val_feat = X_val_feat[top_feature]
X_query_feat = X_query_feat[top_feature]

# Make Predictions idk

In [151]:
models = {"RFC": RandomForestClassifier(), "LDA": LinearDiscriminantAnalysis()}
param_grids = {
            "RFC": [{'n_estimators' : 1000, 'n_jobs' : -1}, 
                    {'n_estimators' : 10000, 'n_jobs' : -1}],
            "LDA": [{'solver' : 'lsqr'},
                    {'solver' : 'eigen'}]
            }

In [152]:
optimum_models = dict()
accuracies = dict()
for model in param_grids:
    classifier = models[model]
    prev_acc = 0
    optimum_param = dict()
    accuracy = []
    for values in param_grids[model]:
        
        #Fitting to the training data with selected hyperparameters
        classifier.set_params(**values)
        print(classifier)
        classifier.fit(X_train_over, y_train_over)

        #Finding the balanced accuracy
        y_hat = classifier.predict(X_val_feat)
        print(y_hat)
        print(y_val)
        balanced_accuracy = balanced_accuracy_score(y_val, y_hat)
        accuracy.append(balanced_accuracy)
        print(accuracy)
        if balanced_accuracy > prev_acc:
            print(prev_acc, balanced_accuracy)
            prev_acc = balanced_accuracy
            optimum_param = values
            
        
    accuracies[model] = accuracy
    optimum_models[model] = classifier.set_params(**optimum_param)

RandomForestClassifier(n_estimators=1000, n_jobs=-1)
[False False False ... False False False]
9609      False
9610      False
9611      False
9612      False
9613      False
          ...  
179187    False
179188    False
179189    False
179190    False
179191    False
Name: connected, Length: 44146, dtype: bool
[0.6584688724148249]
0 0.6584688724148249
RandomForestClassifier(n_estimators=10000, n_jobs=-1)
[False False False ... False False False]
9609      False
9610      False
9611      False
9612      False
9613      False
          ...  
179187    False
179188    False
179189    False
179190    False
179191    False
Name: connected, Length: 44146, dtype: bool
[0.6584688724148249, 0.6552153603942676]
LinearDiscriminantAnalysis(solver='lsqr')
[False False  True ... False False False]
9609      False
9610      False
9611      False
9612      False
9613      False
          ...  
179187    False
179188    False
179189    False
179190    False
179191    False
Name: connected, Length: 4

# Make A Submission

In [154]:
leaderboard_path = "../Data/leaderboard_data.csv"
sub_data = cleaner(leaderboard_path, feature_path, morph_path, submission = True)
sub_data = area_cols(sub_data)

In [155]:
sub_data = one_hot('pre_brain_area', sub_data, '_pre')
sub_data = one_hot('post_brain_area', sub_data, '_post')
sub_data = one_hot('projection_group', sub_data)

In [157]:
sub_data[top_feature]

,adp_dist,post_skeletal_distance_to_soma,pre_skeletal_distance_to_soma,pre_oracle,pre_test_score
0,1.569857,1.066950,-1.520389,0.371616,-0.365468
1,1.072992,1.548930,0.949414,0.371616,-0.365468
2,0.283729,-0.977701,-1.010610,0.371616,-0.365468
3,0.531271,-0.742668,-0.920656,0.371616,-0.365468
4,1.334973,-0.020021,-1.344421,0.371616,-0.365468
...,...,...,...,...,...
42588,-0.620528,-0.368073,-0.917464,0.430932,0.065340
42589,0.073549,0.489645,-0.608341,0.430932,0.065340
42590,0.101911,-0.629141,-1.047202,0.430932,0.065340
42591,0.923462,0.536486,-0.695932,0.430932,0.065340


In [158]:
sub_data["connected"]=optimum_models['LDA'].predict(sub_data[top_feature])

In [160]:
submission_data = sub_data.filter(['ID','connected'])

In [161]:
submission_data.to_csv('submission_data.csv',index=False)